In [1]:
import mo_descriptor as md
import nn_frame as nn
import numpy as np
import subprocess
subprocess.run('export TF_INTRA_OP_PARALLELISM_THREADS=12', shell=True)

2022-07-19 17:29:27.045984: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-19 17:29:27.046262: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


CompletedProcess(args='export TF_INTRA_OP_PARALLELISM_THREADS=12', returncode=0)

In [2]:
'''
prepare data_set
1. make mo_pair descriptor
'''
# x_shift = np.arange(0, 4.1, 0.1)
# y_shift = np.arange(0, 4.1, 0.1)
# z_shift = np.zeros(x_shift.shape)
# # the original mo, e.g. homo
# homo = md.MO_descriptor('data/homo-s0.cube').make()
# lumo = md.MO_descriptor('data/lumo-s0.cube').make()

# # for the original pair of one mo and itself
# homo_pair = md.MO_pair_descriptor(homo, homo).make()
# lumo_pair = md.MO_pair_descriptor(lumo, lumo).make()

# # for the shifted pair
# homo_pairs = np.zeros((len(x_shift)*len(y_shift),) + homo_pair.shape)
# lumo_pairs = np.zeros((len(x_shift)*len(y_shift),) + lumo_pair.shape)
# for ii, i in enumerate(x_shift):
#     for jj, j in enumerate(y_shift):
#         idx = ii * len(y_shift) + jj
#         homo_pairs[idx][0] = homo_pair[0]
#         homo_pairs[idx][1] = np.add(homo_pair[1],i)
#         homo_pairs[idx][2] = np.add(homo_pair[2],j)
#         homo_pairs[idx][3] = homo_pair[3]
#         lumo_pairs[idx][0] = lumo_pair[0]
#         lumo_pairs[idx][1] = np.add(lumo_pair[1],i)
#         lumo_pairs[idx][2] = np.add(lumo_pair[2],j)
#         lumo_pairs[idx][3] = lumo_pair[3]
# np.save('homo_homo_pair.npy', homo_pairs)
# np.save('lumo_lumo_pair.npy', lumo_pairs)
homo_pairs = np.load('homo_homo_pair.npy')
lumo_pairs = np.load('lumo_lumo_pair.npy')

In [3]:
'''
2. read coupling
'''
raw_data = np.loadtxt('data/cdft-V1V2.dat')
c_homo = np.add(raw_data[:,2], raw_data[:,3]) * 1/2
c_lumo = np.add(raw_data[:,4], raw_data[:,5]) * 1/2

In [4]:
train_homo_pairs = homo_pairs
train_lumo_pairs = lumo_pairs[0:1200]

train_c_homo = c_homo
train_c_lumo = c_lumo[0:1200]

test_homo_pairs = homo_pairs[1200:]
test_lumo_pairs = lumo_pairs[1200:]

test_c_homo = c_homo[1200:].reshape((len(c_homo[1200:]),1))
test_c_lumo = c_lumo[1200:]

In [ ]:
setting = {'activation':'tanh', 'nn_shape':(240, 240, 240), 'batch_size':480, 'training_steps':1000000,\
'learning_rate': 0.001, 'decay_rate':0.96, 'decay_per_steps':1000, 'save_step':10000, 'drop_rate':0, 'save_path':'./save/model'}
NN = nn.NN(setting_dict=setting)
NN.train(train_homo_pairs,train_c_homo)
# NN.test(test_homo_pairs, test_c_homo)

2022-07-19 17:29:30.150361: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-19 17:29:30.150411: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-19 17:29:30.150449: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Bai-Group): /proc/driver/nvidia/version does not exist
2022-07-19 17:29:30.151681: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


training step:     0, loss:  1.649343491
training step: 10000, loss:  0.107331477
training step: 20000, loss:  0.087487519
training step: 30000, loss:  0.084894292
training step: 40000, loss:  0.076153010
training step: 50000, loss:  0.058508065
training step: 60000, loss:  0.050011709
training step: 70000, loss:  0.047724072
training step: 80000, loss:  0.047038447
training step: 90000, loss:  0.045836974
training step: 100000, loss:  0.044102628
training step: 110000, loss:  0.043272458
training step: 120000, loss:  0.042994604
training step: 130000, loss:  0.042916823
training step: 140000, loss:  0.042837568
training step: 150000, loss:  0.042811725
training step: 160000, loss:  0.042797539
training step: 170000, loss:  0.042789079
training step: 180000, loss:  0.042785298
training step: 190000, loss:  0.042782791
training step: 200000, loss:  0.042781122
training step: 210000, loss:  0.042780008
training step: 220000, loss:  0.042779662
training step: 230000, loss:  0.042779583


In [ ]:
print((NN.model(test_homo_pairs)-test_c_homo)/test_c_homo)
print(np.mean((NN.model(test_homo_pairs)-test_c_homo)/test_c_homo))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib widget

x = np.linspace(0, 4, 41)
y = np.linspace(0, 4, 41)
X, Y = np.meshgrid(x, y)
Z = NN.model(train_homo_pairs).numpy().reshape((41,41))
Z1 = c_homo.reshape((41,41))
print(Z.shape)

fix, ax = plt.subplots()
ax.contourf(x,y, Z)
ax.set_title('pred')

In [ ]:
fix, ax = plt.subplots()
ax.contourf(x,y, Z)
ax.set_title('pred')

In [ ]:
print(NN.setting)